In [5]:
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from pandas.plotting import scatter_matrix
import sklearn
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
import joblib

In [40]:
def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

In [6]:
pd.set_option('display.max_columns', 200)  # or 1000
pd.set_option('display.max_rows', 200)  # or 1000

In [7]:
DIM = 40
qbasis = np.loadtxt('../data/powerIAEA18480basis.txt')  # shape:(n, M), n=150, M=52*28=1456
basis = qbasis[:DIM, :] # DIM, M

field1 = pd.read_csv('../data/powerIAEA10000.txt', delimiter=' ', header=None, dtype=float)
field2 = pd.read_csv('../data/powerIAEA8480.txt', delimiter=' ', header=None,  dtype=float)
field = pd.concat([field1, field2], ignore_index=True)

inpower18480 = pd.read_csv('../data/inpower18480.out', header=None, delimiter=' ')
coefficient = pd.read_csv('../data/powerIAEA18480coef.txt', header=None, delimiter=' ')
coefficient.shape

SensorofQuarter = np.loadtxt('../data/SensorofQuarter.txt')
observations =  field.dot(SensorofQuarter.T)

In [8]:
inpower= inpower18480.iloc[:, [1, 2, 3, 4]]  # chosing parameters as ML input
inpower.rename(columns = {1:'bu', 2:'st', 3:'pw', 4:'tin'}, inplace=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1276\3460267142.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpower.rename(columns = {1:'bu', 2:'st', 3:'pw', 4:'tin'}, inplace=True)


In [9]:
bins = 20
bu_anchors = np.arange(0 + 615/(2*bins), 600, 615/bins)
st_anchors = np.array([0, 50, 100, 150, 200, 500, 1000, 1500, 2000, 2500])
pw_anchors = np.arange(20 + 40/10, 100, 80/10)
tin_anchors = np.arange(295 + 2.5/10, 300, 5/10)

In [10]:
anchors =[bu_anchors, st_anchors, pw_anchors, tin_anchors]

In [11]:
def  in2c(data, anchors):
    data10  = data.values.repeat(anchors.size).reshape(-1, anchors.size)
    return np.abs(data10 - anchors).argmin(axis = 1)

In [12]:
inpower['bu_c'],inpower['pw_c'], inpower["tin_c"], inpower["st_c"] = \
in2c(inpower['bu'], bu_anchors), in2c(inpower['pw'], pw_anchors), in2c(inpower['tin'], tin_anchors), in2c(inpower['st'], st_anchors)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1276\365056042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpower['bu_c'],inpower['pw_c'], inpower["tin_c"], inpower["st_c"] = \
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1276\365056042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpower['bu_c'],inpower['pw_c'], inpower["tin_c"], inpower["st_c"] = \
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1276\365056042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a c

In [13]:
inpower.head(20)

,bu,st,pw,tin,bu_c,pw_c,tin_c,st_c
0,0,0,61.1084,291.757,0,5,0,0
1,0,0,44.6899,295.345,0,3,0,0
2,0,0,95.8081,291.717,0,9,0,0
3,1,0,76.1768,292.264,0,7,0,0
4,1,0,59.5801,291.247,0,4,0,0
5,1,0,26.7114,293.896,0,0,0,0
6,2,0,42.1777,293.680,0,2,0,0
7,2,0,98.6743,295.354,0,9,0,0
8,2,0,81.2524,296.465,0,7,2,0
9,3,0,81.3696,297.802,0,7,5,0


In [14]:
inpower.describe()

,bu,st,pw,tin,bu_c,pw_c,tin_c,st_c
count,18480.00000,18480.000000,18480.000000,18480.000000,18480.000000,18480.000000,18480.000000,18480.000000
mean,307.50000,800.000000,60.030254,295.001593,9.496753,4.504762,2.242370,4.500000
std,177.82846,861.707712,23.088962,2.878586,5.773752,2.873805,3.024061,2.872359
min,0.00000,0.000000,20.004900,290.000000,0.000000,0.000000,0.000000,0.000000
25%,153.75000,100.000000,39.975600,292.510750,4.750000,2.000000,0.000000,2.000000
50%,307.50000,350.000000,60.450450,294.991000,9.500000,5.000000,0.000000,4.500000
75%,461.25000,1500.000000,80.054325,297.505000,14.250000,7.000000,5.000000,7.000000
max,615.00000,2500.000000,99.995100,299.998000,19.000000,9.000000,9.000000,9.000000


In [15]:
ob_train_full, ob_test,  in_train_full , in_test = train_test_split(observations, inpower, test_size = 0.05, random_state = 42 )
ob_train, ob_val,  in_train , in_val = train_test_split(ob_train_full, in_train_full, test_size = 0.2, random_state = 42 )

In [24]:
pca = PCA(n_components = 0.999)

In [25]:
proc = Pipeline([('pca', pca),
               ('ss', StandardScaler())])

In [26]:
proc.fit(ob_train_full)

Pipeline(steps=[('pca', PCA(n_components=0.999)), ('ss', StandardScaler())])

In [27]:
bu_svc = SVC()

In [28]:
bu_svc.fit(proc.transform(ob_train), in_train['bu_c'])

SVC()

In [29]:
prd = bu_svc.predict(proc.transform(ob_val))

In [30]:
bu_svc.n_features_in_

21

In [39]:
cm_bu=confusion_matrix(in_val['bu_c'], prd)
cm_bu

array([[183,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0, 169,   8,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   3, 174,   4,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   2, 169,   2,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 171,   5,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 170,   1,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 162,   5,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   1, 172,   3,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 179,   1,   0,  

In [42]:
print(bmatrix(cm_bu))

\begin{bmatrix}
  183 & 2 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0\\
  0 & 169 & 8 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0\\
  0 & 3 & 174 & 4 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0\\
  0 & 0 & 2 & 169 & 2 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0\\
  0 & 0 & 0 & 0 & 171 & 5 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0\\
  0 & 0 & 0 & 0 & 0 & 170 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 162 & 5 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 1 & 172 & 3 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 179 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 173 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 3 & 181 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 

In [32]:
f1_score(in_val['bu_c'], prd, average='macro')

0.9837080547508869

In [33]:
st_svc = SVC()
st_svc.fit(proc.transform(ob_train), in_train['st_c'])
st_prd = st_svc.predict(proc.transform(ob_val))
confusion_matrix(in_val['st_c'], st_prd)

array([[357,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  7, 314,   6,   0,   0,   0,   0,   0,   0,   0],
       [  6,   2, 337,  15,   0,   0,   0,   0,   0,   0],
       [  0,   0,  15, 327,  20,   0,   0,   0,   0,   0],
       [  1,   0,   0,  15, 349,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 354,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 320,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0, 376,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 334,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 357]], dtype=int64)

In [34]:
f1_score(in_val['st_c'], st_prd, average='macro')

0.9756710084926434

In [38]:
st_svc.n_features_in_

21

In [35]:
in_val.iloc[1]

bu        416.0000
st       2500.0000
pw         31.1987
tin       292.9200
bu_c       13.0000
pw_c        1.0000
tin_c       0.0000
st_c        9.0000
Name: 17880, dtype: float64

In [36]:
pw_svc = SVC()
pw_svc.fit(proc.transform(ob_train), in_train['pw_c'])
pw_prd = pw_svc.predict(proc.transform(ob_val))
confusion_matrix(in_val['pw_c'], pw_prd)

array([[237, 108,  26,   6,   0,   0,   0,   0,   0,   0],
       [ 76, 165,  57,  28,   5,   0,   0,   0,   0,   0],
       [ 13,  87, 138,  75,   9,  12,   1,   0,   0,   0],
       [  2,  24,  83, 155,  63,  31,   5,   2,   0,   0],
       [  0,   2,  27,  84,  87, 105,  27,   6,   0,   0],
       [  0,   0,   1,  31,  58, 165,  86,  36,  10,   0],
       [  0,   0,   0,   0,  10, 106, 130,  50,  30,  12],
       [  0,   0,   0,   0,   0,  31,  80, 101,  96,  37],
       [  0,   0,   0,   0,   0,   4,  27,  69, 177,  95],
       [  0,   0,   0,   0,   0,   0,   2,  16, 118, 188]], dtype=int64)

In [37]:
f1_score(in_val['pw_c'], pw_prd, average="macro")

0.4371485185963895

In [24]:
class MultiPredictor (BaseEstimator, TransformerMixin):
    def __init__(self, predictors):
        self.predictors = predictors
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X
    def predict(self, X):
        y =np.array([e.predict(X) for e in self.predictors])
        return y.T.squeeze()
    def decision_function(self, X):
        y =np.array([e.decision_function(X) for e in self.predictors]).squeeze()
        return y

In [25]:
obs2in = Pipeline([('proc', proc),
                   ('multipred', MultiPredictor([bu_svc, st_svc, pw_svc]))
                  ])

In [26]:
y = obs2in.predict(ob_val.iloc[1:2,:])

In [27]:
anchors

[array([ 15.375,  46.125,  76.875, 107.625, 138.375, 169.125, 199.875,
        230.625, 261.375, 292.125, 322.875, 353.625, 384.375, 415.125,
        445.875, 476.625, 507.375, 538.125, 568.875, 599.625]),
 array([   0,   50,  100,  150,  200,  500, 1000, 1500, 2000, 2500]),
 array([24., 32., 40., 48., 56., 64., 72., 80., 88., 96.]),
 array([295.25, 295.75, 296.25, 296.75, 297.25, 297.75, 298.25, 298.75,
        299.25, 299.75])]

In [28]:
def get_bound____(cs, anchors, ms):
    # cs.shape:1*3, ms.shape:1*3, anchors.shape:1*3
    return [(a[max(0, c-m)], a[min(a.size-1, c+m)])  for c, a, m in zip(cs, anchors, ms)]

In [29]:
# 原论文中的6个测试样本的输入
mus = np.array([[0, 0, 61.11, 291.76],
                [200, 500, 73.51, 290.54],
                [400, 1000, 28.03, 298.74],
                [500, 1000, 52.96, 291.51],
                [130, 1500, 52.15, 291.01],
                [130, 1500, 52.15, 291.01]])

In [30]:
# 原论文中6个测试输入的猜测值
mu_initials = np.array([[20, 100, 61.11, 291.76],
                      [210, 600, 71.51, 293.54],
                      [400, 1000, 18.00, 293.54],
                      [500, 1000, 35.00, 297.54],
                      [120, 1700, 52.15, 300.00],
                      [130, 1480, 52.15, 300.00]])

In [31]:
#  6个测试输入在数据集中的编号
mus_indx = [(inpower.iloc[:,0:4] - mus[i]).abs().sum(axis = 1).argmin() for i in range(6)]

In [32]:
mus_indx

[0, 9841, 12288, 12589, 13326, 13326]

In [33]:
x = observations.iloc[mus_indx,:]

c = obs2in.predict(x)

to_anchor = lambda y: np.array([ a[c]  for a , c in zip(anchors, y.T)]).T

y = to_anchor(c)

y, c

(array([[  15.375,  150.   ,   64.   ],
        [ 199.875,  500.   ,   72.   ],
        [ 384.375, 1000.   ,   64.   ],
        [ 507.375, 1000.   ,   64.   ],
        [ 138.375, 1500.   ,   40.   ],
        [ 138.375, 1500.   ,   40.   ]]),
 array([[ 0,  3,  5],
        [ 6,  5,  6],
        [12,  6,  5],
        [16,  6,  5],
        [ 4,  7,  2],
        [ 4,  7,  2]], dtype=int64))

In [34]:
limits = np.array([[0,615],[0, 2500], [20, 100], [290, 300]])
def get_bounds(ys,  offsets = [45, 300, 50], limits = limits):
    # cs.shape:1*3, ms.shape:1*3, anchors.shape:1*3
    return [(max(l[0], y - o),  min(l[1], y +o) )  for y, l, o in zip(ys, limits[0:3], offsets)] 

In [35]:
# 确定搜索空间
get_bounds(y[0]) + [(290, 300)]

[(0, 60.375), (0, 450.0), (20, 100), (290, 300)]

In [37]:
knn = joblib.load('knn4.pkl')
predict_KNN = knn.predict(inpower.iloc[:,0:4])  
predict_KNN.shape

(18480, 40)

In [ ]:
knn = joblib.load('knn4.pkl')
predict_KNN = knn.predict(inpower)  
predict_KNN.shape

In [40]:
sklearn.__version__

'0.24.2'